## Mini Project 2: Consumer Complaint Resolution Analysis Using Python

Tasks to be performed:

The following tasks are to be performed:Note:Please completethe given steps on both train and test data.•Read the Data from the Given excel file.•Check the data type for both data (test file and train file)•Do missing value analysis and dropcolumns where more than 25% of data are missing •Extracting Day, Month,and Year from Date Received Column and create new fields for a month, year,and day •Calculatethe Number ofDays the Complaint was with the Companyand create a new field as “Days held”•Drop "Date Received","Date Sent to Company","ZIP Code", "Complaint ID"fields•Imputing Nullvalue in “State”by Mode•with the help ofthe days we calculated above,create a newfield 'Week_Received'where we calculate the week based on the day of receiving.•store data of disputed people into the “disputed_cons” variable for future tasks •Plot bar graph of thetotal no of disputes of consumers with the help ofseaborn•Plot bar graph of thetotal no of disputes products-wise with the help ofseaborn•Plot bar graph ofthe total no of disputes with Top Issues by Highest Disputes, with the help ofseaborn•Plot bar graph ofthe total no of disputes by State with Maximum Disputes•Plotbar graph ofthe total no of disputes Submitted Via differentsource•Plot bar graph ofthe total no of disputes where the Company's Response to the Complaints•Plot bar graph ofthe total no of disputes where the Company's Response Leadsto Disputes•Plot bar graph ofthe total no of disputes.Whether there are Disputes Instead of Timely Response•Plot bar graph ofthe total no of disputes over Year Wise Complaints•Plot bar graph ofthe total no of disputes over Year Wise Disputes•Plot bar graph ofTop Companies with Highest Complaints•Converte all negative days held to zero(it is the time taken by the authority that can't be negative)•Drop UnnecessaryColumns for the Model Buildinglike:'Company', 'State', 'Year_Received', 'Days_held'•Change Consumer Disputed Column to 0 and 1(yes to 1, and no to 0)•Create Dummy Variables for categoricalfeaturesand concat with the original data framelike: 'Product,’'Submitted via,’'Company response to consumer,’'Timely response?'•Scaling the Data Sets (note:discard dependent variable before doing standardization)and Makefeature Selection with the help ofPCAup to 80% of the information.•Splittingthe Data Sets Into X and Y by the dependent and independent variables (data selected by PCA)•Build given models andmeasuretheir test and validationaccuracy:oLogisticRegressionoDecisionTreeClassifieroRandomForestClassifieroAdaBoostClassifieroGradientBoostingClassifieroKNeighborsClassifieroXGBClassifier•Whoever gives the most accurate result uses it and predicts the outcomefor the test file and fills its dispute column so the business team can take some action accordingly.


In [1]:
# Task 1: Importing necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import datetime as dt

In [2]:
# Task 2: Read the Data from the Given excel file.
train_data=pd.read_csv("C:\\Users\\King\\Desktop\\DSML Internship\\Assignments\\Mini Project (ML)\\Datasets\\Consumer_Complaints_train.csv")
test_data=pd.read_csv("C:\\Users\\King\\Desktop\\DSML Internship\\Assignments\\Mini Project (ML)\\Datasets\\Consumer_Complaints_test.csv")
train_data.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2015-10-14,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,NaN,NaN,Equifax,GA,30134,NaN,Consent not provided,Web,2015-10-14,Closed with explanation,Yes,No,1605653
1,2015-04-26,Bank account or service,Other bank product/service,Deposits and withdrawals,NaN,RE : XXXX XXXX XXXX-PRIVILEGED AND CONFIDENTIA...,NaN,Wells Fargo & Company,GA,319XX,NaN,Consent provided,Web,2015-04-26,Closed with explanation,Yes,Yes,1347613
2,2013-12-20,Credit card,NaN,Other,NaN,NaN,NaN,Citibank,SC,29203,NaN,NaN,Phone,2014-01-03,Closed with non-monetary relief,Yes,No,640394
3,2016-03-03,Debt collection,"Other (i.e. phone, health club, etc.)",Disclosure verification of debt,Not given enough info to verify debt,NaN,Company has responded to the consumer and the ...,"FAIR COLLECTIONS & OUTSOURCING, INC.",OH,43082,NaN,NaN,Referral,2016-03-04,Closed with explanation,Yes,No,1815134
4,2015-01-30,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,NaN,NaN,"HCFS Health Care Financial Services, Inc.",CA,90036,NaN,NaN,Web,2015-01-30,Closed with explanation,Yes,Yes,1218613


In [3]:
test_data.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Complaint ID
0,2015-01-17,Credit card,NaN,Customer service / Customer relations,NaN,NaN,NaN,Citibank,TX,75241,NaN,NaN,Web,2015-01-17,Closed with monetary relief,Yes,1198133
1,2016-06-22,Consumer Loan,Title loan,Payment to acct not credited,NaN,NaN,Company believes it acted appropriately as aut...,"Larsen MacColl Partners II, L.P.",TX,76548,Servicemember,NaN,Phone,2016-06-22,Closed with explanation,Yes,1979989
2,2015-09-04,Credit card,NaN,Credit line increase/decrease,NaN,I WANT TO REQUEST A CREDIT LINE INCREASE OF XX...,NaN,Capital One,NC,271XX,NaN,Consent provided,Web,2015-09-04,Closed with explanation,Yes,1552090
3,2016-05-17,Consumer Loan,Installment loan,Problems when you are unable to pay,NaN,I have asked One Main Financial not to call my...,NaN,"OneMain Financial Holdings, LLC",MO,634XX,NaN,Consent provided,Web,2016-05-20,Closed with non-monetary relief,Yes,1929306
4,2016-07-07,Debt collection,"Other (i.e. phone, health club, etc.)",Improper contact or sharing of info,Contacted employer after asked not to,I have received several calls from a XXXX XXXX...,Company has responded to the consumer and the ...,"GMA Investments, LLC",SC,296XX,NaN,Consent provided,Web,2016-07-07,Closed with explanation,Yes,2001667


In [4]:
# Task 3: Check the data type for both data (test file and train file)
print("Train Data Types:")
print(train_data.dtypes)
print("\nTest Data Types:")
print(test_data.dtypes)

Train Data Types:
Date received                   object
Product                         object
Sub-product                     object
Issue                           object
Sub-issue                       object
Consumer complaint narrative    object
Company public response         object
Company                         object
State                           object
ZIP code                        object
Tags                            object
Consumer consent provided?      object
Submitted via                   object
Date sent to company            object
Company response to consumer    object
Timely response?                object
Consumer disputed?              object
Complaint ID                     int64
dtype: object

Test Data Types:
Date received                   object
Product                         object
Sub-product                     object
Issue                           object
Sub-issue                       object
Consumer complaint narrative    object
Company public

In [5]:
# Missing value analysis for train data
missing_train = train_data.isnull().sum()  # Count missing values in each column
missing_train_percentage = (missing_train / len(train_data)) * 100  # Calculate percentage

# Missing value analysis for test data
missing_test = test_data.isnull().sum()  # Count missing values in each column
missing_test_percentage = (missing_test / len(test_data)) * 100  # Calculate percentage

In [6]:
# Task 4: Do missing value analysis and drop columns where more than 25% of data are missing
# Display columns where more than 25% of data are missing in train data
columns_to_drop_train = missing_train_percentage[missing_train_percentage > 25].index
print("Columns to drop in train data due to missing values:")
print(columns_to_drop_train)

# Drop columns in train data where more than 25% of data are missing
train_data = train_data.drop(columns=columns_to_drop_train)

# Display columns where more than 25% of data are missing in test data
columns_to_drop_test = missing_test_percentage[missing_test_percentage > 25].index
print("\nColumns to drop in test data due to missing values:")
print(columns_to_drop_test)

# Drop columns in test data where more than 25% of data are missing
test_data = test_data.drop(columns=columns_to_drop_test)

Columns to drop in train data due to missing values:
Index(['Sub-product', 'Sub-issue', 'Consumer complaint narrative',
       'Company public response', 'Tags', 'Consumer consent provided?'],
      dtype='object')

Columns to drop in test data due to missing values:
Index(['Sub-product', 'Sub-issue', 'Consumer complaint narrative',
       'Company public response', 'Tags', 'Consumer consent provided?'],
      dtype='object')


In [7]:
# Task 5: Extracting Day, Month,and Year from Date Received Column and create new fields for a month, year,and day 
# Convert 'Date Received' column to datetime format
train_data['Date received'] = pd.to_datetime(train_data['Date received'])
test_data['Date received'] = pd.to_datetime(test_data['Date received'])

# Extract day, month, and year into new columns in train data
train_data['Day'] = train_data['Date received'].dt.day
train_data['Month'] = train_data['Date received'].dt.month
train_data['Year'] = train_data['Date received'].dt.year

# Extract day, month, and year into new columns in test data
test_data['Day'] = test_data['Date received'].dt.day
test_data['Month'] = test_data['Date received'].dt.month
test_data['Year'] = test_data['Date received'].dt.year

# Display the updated train and test data with new columns for day, month, and year
print("Train Data with New Columns:")
print(train_data.head())

print("\nTest Data with New Columns:")
print(test_data.head())

Train Data with New Columns:
  Date received                  Product  \
0    2015-10-14         Credit reporting   
1    2015-04-26  Bank account or service   
2    2013-12-20              Credit card   
3    2016-03-03          Debt collection   
4    2015-01-30          Debt collection   

                                    Issue  \
0  Incorrect information on credit report   
1                Deposits and withdrawals   
2                                   Other   
3         Disclosure verification of debt   
4         Disclosure verification of debt   

                                     Company State ZIP code Submitted via  \
0                                    Equifax    GA    30134           Web   
1                      Wells Fargo & Company    GA    319XX           Web   
2                                   Citibank    SC    29203         Phone   
3       FAIR COLLECTIONS & OUTSOURCING, INC.    OH    43082      Referral   
4  HCFS Health Care Financial Services, Inc.    CA

In [8]:
# Task 6: Calculate the Number of Days the Complaint was with the Company and create a new field as “Days held”
# Convert 'Date Received' and 'Date Sent to Company' columns to datetime format
train_data['Date received'] = pd.to_datetime(train_data['Date received'])
train_data['Date sent to company'] = pd.to_datetime(train_data['Date sent to company'])

test_data['Date received'] = pd.to_datetime(test_data['Date received'])
test_data['Date sent to company'] = pd.to_datetime(test_data['Date sent to company'])

# Calculate the number of days the complaint was with the company for train data
train_data['Days held'] = (train_data['Date sent to company'] - train_data['Date received']).dt.days

# Calculate the number of days the complaint was with the company for test data
test_data['Days held'] = (test_data['Date sent to company'] - test_data['Date received']).dt.days

# Display the updated train and test data with the new 'Days held' field
print("Train Data with 'Days held' field:")
print(train_data.head())

print("\nTest Data with 'Days held' field:")
print(test_data.head())


Train Data with 'Days held' field:
  Date received                  Product  \
0    2015-10-14         Credit reporting   
1    2015-04-26  Bank account or service   
2    2013-12-20              Credit card   
3    2016-03-03          Debt collection   
4    2015-01-30          Debt collection   

                                    Issue  \
0  Incorrect information on credit report   
1                Deposits and withdrawals   
2                                   Other   
3         Disclosure verification of debt   
4         Disclosure verification of debt   

                                     Company State ZIP code Submitted via  \
0                                    Equifax    GA    30134           Web   
1                      Wells Fargo & Company    GA    319XX           Web   
2                                   Citibank    SC    29203         Phone   
3       FAIR COLLECTIONS & OUTSOURCING, INC.    OH    43082      Referral   
4  HCFS Health Care Financial Services, Inc.

In [9]:
# Task 7: Drop "Date Received","Date Sent to Company","ZIP Code", "Complaint ID" fields
# Columns to drop
columns_to_drop = ['Date received', 'Date sent to company', 'ZIP code', 'Complaint ID']

# Drop specified columns from train and test data
train_data.drop(columns=columns_to_drop, inplace=True)
test_data.drop(columns=columns_to_drop, inplace=True)

# Impute null values in 'State' column with the mode (most frequent value) for both train and test data
train_data['State'].fillna(train_data['State'].mode()[0], inplace=True)
test_data['State'].fillna(test_data['State'].mode()[0], inplace=True)

# Display the updated train and test data after dropping columns and imputing null values in 'State'
print("Train Data after dropping columns and imputing 'State' null values:")
print(train_data.head())

print("\nTest Data after dropping columns and imputing 'State' null values:")
print(test_data.head())

Train Data after dropping columns and imputing 'State' null values:
                   Product                                   Issue  \
0         Credit reporting  Incorrect information on credit report   
1  Bank account or service                Deposits and withdrawals   
2              Credit card                                   Other   
3          Debt collection         Disclosure verification of debt   
4          Debt collection         Disclosure verification of debt   

                                     Company State Submitted via  \
0                                    Equifax    GA           Web   
1                      Wells Fargo & Company    GA           Web   
2                                   Citibank    SC         Phone   
3       FAIR COLLECTIONS & OUTSOURCING, INC.    OH      Referral   
4  HCFS Health Care Financial Services, Inc.    CA           Web   

      Company response to consumer Timely response? Consumer disputed?  Day  \
0          Closed with 

In [10]:
# Task 8: Imputing Null value in “State” by Mode
train_data['State'].fillna(train_data['State'].mode()[0], inplace=True)
test_data['State'].fillna(test_data['State'].mode()[0], inplace=True)

In [11]:
# Task 9: with the help of the days we calculated above,create a new field 'Week_Received' where we calculate the week based on the day of receiving.
train_data['Week_Received'] = train_data['Date received'].dt.week
test_data['Week_Received'] = test_data['Date Received'].dt.week

KeyError: 'Date received'

In [ ]:
# Task 10: store data of disputed people into the “disputed_cons” variable for future tasks
disputed_cons = train_data[train_data['Consumer disputed?'] == 'Yes']

In [ ]:
# Task 11: Plot bar graph of the total no of disputes of consumers with the help of seaborn
plt.figure(figsize=(10, 6))
sns.countplot(x='Consumer disputed?', data=train_data)
plt.title('Total Number of Disputes')
plt.show()

In [ ]:
# Task 12: Plot bar graph of the total no of disputes products-wise with the help of seaborn
plt.figure(figsize=(12, 6))
sns.countplot(x='Product', hue='Consumer disputed?', data=train_data)
plt.title('Total Number of Disputes Products-wise')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Task 13: Plot bar graph of the total no of disputes with Top Issues by Highest Disputes, with the help of seaborn
top_issues = train_data['Issue'].value_counts().nlargest(10).index
plt.figure(figsize=(14, 6))
sns.countplot(x='Issue', hue='Consumer disputed?', data=train_data[train_data['Issue'].isin(top_issues)])
plt.title('Total Number of Disputes with Top Issues')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Task 14: Plot bar graph of the total no of disputes by State with Maximum Disputes
plt.figure(figsize=(16, 6))
sns.countplot(x='State', hue='Consumer disputed?', data=train_data)
plt.title('Total Number of Disputes by State')
plt.show()

In [ ]:
# Task 15: Plot bar graph of the total no of disputes Submitted Via different source
plt.figure(figsize=(10, 6))
sns.countplot(x='Submitted via', hue='Consumer disputed?', data=train_data)
plt.title('Total Number of Disputes Submitted Via')
plt.show()

In [ ]:
# Task 16: Plot bar graph of the total no of disputes where the Company's Response to the Complaints
plt.figure(figsize=(10, 6))
sns.countplot(x='Company response to consumer', hue='Consumer disputed?', data=train_data)
plt.title("Total Number of Disputes where Company's Response to the Complaints")
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Task 17: Plot bar graph of the total no of disputes where the Company's Response Leads to Disputes
plt.figure(figsize=(10, 6))
sns.countplot(x='Company response to consumer', hue='Consumer disputed?', data=disputed_cons)
plt.title("Total Number of Disputes where Company's Response Leads to Disputes")
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Task 18: Plot bar graph of the total no of disputes.Whether there are Disputes Instead of Timely Response
plt.figure(figsize=(10, 6))
sns.countplot(x='Timely response?', hue='Consumer disputed?', data=train_data)
plt.title('Total Number of Disputes Instead of Timely Response')
plt.show()

In [ ]:
# Task 19: Plot bar graph of the total no of disputes over Year Wise Complaints
plt.figure(figsize=(10, 6))
sns.countplot(x='Year_Received', hue='Consumer disputed?', data=train_data)
plt.title('Total Number of Disputes over Year Wise Complaints')
plt.show()

In [ ]:
# Task 20: Plot bar graph of the total no of disputes over Year Wise Disputes
plt.figure(figsize=(10, 6))
sns.countplot(x='Year_Received', hue='Consumer disputed?', data=disputed_cons)
plt.title('Total Number of Disputes over Year Wise Complaints')
plt.show()

In [ ]:
# Task 21: Plot bar graph of Top Companies with Highest Complaints
top_companies = train_data['Company'].value_counts().nlargest(10).index
plt.figure(figsize=(14, 6))
sns.countplot(x='Company', hue='Consumer disputed?', data=train_data[train_data['Company'].isin(top_companies)])
plt.title('Top Companies with Highest Complaints')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Task 22: Converte all negative days held to zero(it is the time taken by the authority that can't be negative)
train_data['Days_held'] = train_data['Days_held'].apply(lambda x: max(x, 0))

In [ ]:
# Task 23: Drop Unnecessary Columns for the Model Building like:'Company', 'State', 'Year_Received', 'Days_held'
columns_to_drop_model = ['Company', 'State', 'Year_Received', 'Days_held']
train_data_model = train_data.drop(columns=columns_to_drop_model)

In [ ]:
# Task 24: Change Consumer Disputed Column to 0 and 1(yes to 1, and no to 0)
train_data_model['Consumer disputed?'] = (train_data_model['Consumer disputed?'] == 'Yes').astype(int)

In [ ]:
# Task 25: Create Dummy Variables for categorical features and concat with the original data frame like: 'Product,’'Submitted via,’'Company response to consumer,’'Timely response?'
categorical_cols = ['Product', 'Submitted via', 'Company response to consumer', 'Timely response?']
train_data = pd.get_dummies(train_data, columns=categorical_cols, drop_first=True)

In [ ]:
# Task 26: Scaling the Data Sets (note:discard dependent variable before doing standardization)and Make feature Selection with the help of PCA up to 80% of the information.
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA

# Selecting features and target
X = train_data.drop(columns=['Consumer disputed?'])
y = train_data['Consumer disputed?']

# Encode categorical variables using LabelEncoder
label_encoder = LabelEncoder()
categorical_cols = X.select_dtypes(include=['object']).columns
X[categorical_cols] = X[categorical_cols].apply(label_encoder.fit_transform)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=0.8)
X_pca = pca.fit_transform(X_scaled)


In [ ]:
# Task 27: Splitting the Data Sets Into X and Y by the dependent and independent variables (data selected by PCA)
X_train, X_val, y_train, y_val = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [ ]:
# Task 28: Build given models and measure their test and validation accuracy:LogisticRegressionon DecisionTreeClassifier RandomForestClassifier AdaBoostClassifier GradientBoostingClassifier KNeighborsClassifier XGBClassifier
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    #'Random Forest': RandomForestClassifier(),
    #'AdaBoost': AdaBoostClassifier(),
    #'Gradient Boosting': GradientBoostingClassifier(),
    #'K Neighbors': KNeighborsClassifier(),
    #'XGBoost': XGBClassifier()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    train_accuracy = accuracy_score(y_train, y_pred_train)
    val_accuracy = accuracy_score(y_val, y_pred_val)
    print(f'{name} - Train Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}')


In [ ]:
best_model = LogisticRegression()  # Replace with the best-performing model
best_model.fit(X_train, y_train)

In [ ]:
test_data = pd.get_dummies(test_data, columns=categorical_columns, drop_first=True)

In [ ]:
test_data_scaled = scaler.transform(test_data)
test_data_pca = pca.transform(test_data_scaled)

In [ ]:
test_predictions = best_model.predict(test_data_pca)

In [ ]:
test_data['Consumer disputed?'] = test_predictions

In [ ]:
test_data.to_csv('predicted_test_file.csv', index=False)